In [1]:
import mysql.connector
from mysql.connector import errorcode

In [3]:
import config2

In [2]:
DB_NAME = 'yelp_info'

In [4]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [9]:
# cursor.execute("""CREATE DATABASE yelp_info;""")

In [15]:
# cursor.execute("""CREATE TABLE businesses
                (id varchar(255) NOT NULL,
                name text NOT NULL,
                review_count int NOT NULL,
                rating float NOT NULL,
                price varchar(255) NOT NULL,
                city text NOT NULL,
                PRIMARY KEY (id));
                """)

In [6]:
 cursor.execute("""CREATE TABLE reviews
                (review_id varchar(255) NOT NULL,
                text text NOT NULL,
                review_rating float NOT NULL,
                date datetime NOT NULL,
                bus_id varchar(255) NOT NULL,
                PRIMARY KEY (review_id)
                );
                """)

### STEP 2: Yelp API Stuff

Create Python functions to:

Perform a search of businesses using pagination

Parse the API response for specific data points

Insert the data into your AWS DB

Use the functions above in a loop that will paginate over the results to retrieve all of the results.

In [ ]:
#write a finction to make a call to the DB

In [7]:
import pandas as pd

In [8]:
import requests

In [1]:
import time


ModuleNotFoundError: No module named 'urllib2'

In [10]:
client_id = 'inkVgew63NKO9Oi_a7ADyw'
api_key = 'MO23Q20e_W12VdbSHXy2-apAWZgU22unGpFEOc31J0HdZrTTry78gzzEBdBXZz5N6Bil5HNb3NQDlf4Hl72hWzOxloc1VWFoVjhVjDudve_Kezq3MRdrkzKzjGZ6XXYx'

In [11]:
term = 'Brewery'
location = 'Brooklyn NY'
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': 50,
                'offset': 0
            }
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [12]:
#CODE I'M WORKING ON
def yelp_call_businesses(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    return data

def get_all_results(url_params, api_key):
    offset = 0
    total_bus = []
    while offset < 1000:
        url_params['offset'] = offset
        print(url_params)
        total_bus.append(yelp_call_businesses(url_params, api_key))
        offset += 50
    return total_bus

In [15]:
def yelp_call_reviews(bus_id):
    url = f'https://api.yelp.com/v3/businesses/{bus_id}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers)
    data = response.json()
    return data

In [14]:
def get_all_reviews(restaurant_list_new):
    review_list = []
    for restaurant in restaurant_list_new:
        my_dict = {}
        my_dict['rest_id'] = restaurant['id']
        my_dict['review'] = yelp_call_reviews(restaurant['id'])
        review_list.append(dict(my_dict))
    return review_list    

In [16]:
test_yelp_data = get_all_results(url_params, api_key)

{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 0}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 50}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 100}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 150}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 200}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 250}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 300}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 350}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 400}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 450}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 500}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 550}
{'term': 'Brewery', 'location': 'Brooklyn+NY', 'limit': 50, 'offset': 600}
{'term': 'Brewery', 'locatio

In [93]:
test_yelp_data[0]

{'businesses': [{'id': 'gUu-0hyujy5xRM-JAty9JQ',
   'alias': 'other-half-brewing-brooklyn',
   'name': 'Other Half Brewing',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/yC8t4IPXrsfcvSypBHS_zw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=inkVgew63NKO9Oi_a7ADyw',
   'review_count': 323,
   'categories': [{'alias': 'breweries', 'title': 'Breweries'}],
   'rating': 4.0,
   'coordinates': {'latitude': 40.673875, 'longitude': -73.999095},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '195 Centre St',
    'address2': '',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11231',
    'country': 'US',
    'state': 'NY',
    'display_address': ['195 Centre St', 'Brooklyn, NY 11231']},
   'phone': '+19177656107',
   'display_phone': '(917) 765-6107',
   'distance': 5578.9111303012305},
  {'id': 

In [17]:
restaurant_list_new = []
for data in test_yelp_data:
    print(data)
    for restaurant in data['businesses']:
        restaurant_info = {}
        restaurant_info['id'] = restaurant['id']
        restaurant_info['name'] = restaurant['name']
        restaurant_info['review_count'] = restaurant['review_count']
        restaurant_info['rating'] = restaurant['rating']
        if 'price' not in restaurant.keys():
            restaurant_info['price'] = None
        else:
            restaurant_info['price'] = restaurant['price']
        restaurant_info['city'] = restaurant['location']['city']
        restaurant_list_new.append(restaurant_info)

{'businesses': [{'id': 'gUu-0hyujy5xRM-JAty9JQ', 'alias': 'other-half-brewing-brooklyn', 'name': 'Other Half Brewing', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/yC8t4IPXrsfcvSypBHS_zw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=inkVgew63NKO9Oi_a7ADyw', 'review_count': 323, 'categories': [{'alias': 'breweries', 'title': 'Breweries'}], 'rating': 4.0, 'coordinates': {'latitude': 40.673875, 'longitude': -73.999095}, 'transactions': [], 'price': '$$', 'location': {'address1': '195 Centre St', 'address2': '', 'address3': '', 'city': 'Brooklyn', 'zip_code': '11231', 'country': 'US', 'state': 'NY', 'display_address': ['195 Centre St', 'Brooklyn, NY 11231']}, 'phone': '+19177656107', 'display_phone': '(917) 765-6107', 'distance': 5578.9111303012305}, {'id': 'GBg3nixNLJlxMBypnuNTwQ', 'alias': 'sixpoint-brewery-brooklyn', 'name': 'Si

In [58]:
restaurant_list_new[0]

{'id': 'gUu-0hyujy5xRM-JAty9JQ',
 'name': 'Other Half Brewing',
 'review_count': 323,
 'rating': 4.0,
 'price': '$$',
 'city': 'Brooklyn'}

In [97]:
rest_tuples = []
for restaurant in restaurant_list_new:
    restaurant_tuple = (restaurant['id'], restaurant['name'], restaurant['review_count'], restaurant['rating'], restaurant['price'], restaurant['city'])
    rest_tuples.append(restaurant_tuple)

In [98]:
rest_tuples

[('gUu-0hyujy5xRM-JAty9JQ', 'Other Half Brewing', 323, 4.0, '$$', 'Brooklyn'),
 ('hKChLzaG8tyrSrg3U_wRNw', 'Threes Brewing', 358, 4.0, '$$', 'Brooklyn'),
 ('GBg3nixNLJlxMBypnuNTwQ', 'Sixpoint Brewery', 32, 4.5, '$$', 'Brooklyn'),
 ('oFDKXoHhSBQgDGjADLfQEg',
  'The Brooklyn Brewery',
  950,
  4.0,
  '$$',
  'Brooklyn'),
 ('sXe7zhFps8ztADuiFAHF5w', 'Grimm Artisanal Ales', 67, 4.0, '$$', 'Brooklyn'),
 ('oaK9qq1lH59AdH44z_Lysw',
  'Five Boroughs Brewing Co',
  72,
  4.5,
  '$$',
  'Brooklyn'),
 ('ldOFPtan_IrjtM70TivhMA', 'Strong Rope Brewery', 78, 4.5, '$$', 'Brooklyn'),
 ('iNvDnRFJWuoO3asZOta2kw',
  'Kings County Brewers Collective',
  98,
  4.5,
  '$$',
  'Brooklyn'),
 ('-7PnG_cD9VY-IfHGWzynmQ', 'Ainslie', 4, 5.0, None, 'Brooklyn'),
 ('7gH6Gcf8tnLlctw9kzKNHw',
  'Bridge and Tunnel Brewery',
  45,
  4.5,
  '$',
  'Ridgewood'),
 ('ACduxcM6wdaJPVHKy9GHbQ', 'Circa Brewing', 270, 4.0, '$$', 'Brooklyn'),
 ('RhtO1FSefPfqpojBAwuEWw', 'Transmitter Brewing', 80, 4.5, '$$', 'Brooklyn'),
 ('RpRvCAzL

In [100]:
stmt = """INSERT INTO businesses (id, name, review_count, rating, price, city) 
            VALUES (%s, %s, %s, %s, %s, %s)"""

cursor.executemany(stmt, rest_tuples)
cnx.commit()
cursor.close()
cnx.close()

In [18]:
yelp_reviews = get_all_reviews(restaurant_list_new)

In [112]:
#for restaurant in restaurant_list_new:
#     reviews = yelp_call_reviews(restaurant['id'])
#     print(reviews)

{'reviews': [{'id': '8g0IHHggKE2Bq-RNSS283Q', 'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=8g0IHHggKE2Bq-RNSS283Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'Love this hidden, off the beaten path gem of a brewery. It gets packed on the weekends and is super lively, but in the best Brooklyn-vibes kind of way....', 'rating': 5, 'time_created': '2019-09-07 09:32:40', 'user': {'id': 'PrioZ6UL7liEWQRmNtgq6w', 'profile_url': 'https://www.yelp.com/user_details?userid=PrioZ6UL7liEWQRmNtgq6w', 'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/d96R823jwSwBCBLILpA1mA/o.jpg', 'name': 'Kaley R.'}}, {'id': '2MISSNscaoUdpw0nAgMPZg', 'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=2MISSNscaoUdpw0nAgMPZg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Hallelujah!! I ha

{'reviews': [{'id': 'BWcjdIYWrd21T9Tg4HrUIA', 'url': 'https://www.yelp.com/biz/grimm-artisanal-ales-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=BWcjdIYWrd21T9Tg4HrUIA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "The nomadic brewery finally payed down roots via a Brooklyn taproom. Well if you already didn't know some of the best sours in the east coast come from...", 'rating': 5, 'time_created': '2019-07-21 19:35:53', 'user': {'id': 'ukW-L1dza6CxQwxe6G7b2Q', 'profile_url': 'https://www.yelp.com/user_details?userid=ukW-L1dza6CxQwxe6G7b2Q', 'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/YDOw3Fc69M6LJfBH4iLeBw/o.jpg', 'name': 'Eric L.'}}, {'id': '8ByQAhmEHiXyRPwFhg9H0A', 'url': 'https://www.yelp.com/biz/grimm-artisanal-ales-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=8ByQAhmEHiXyRPwFhg9H0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'Saturday late 

{'reviews': [{'id': 'UoFtSyMhSvpOiSQlo4loag', 'url': 'https://www.yelp.com/biz/ainslie-brooklyn-2?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=UoFtSyMhSvpOiSQlo4loag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'Our experience at the Ainslie was mostly fantastic - the food was incredible, our service (by Evan) was impeccable, and the wine was delicious. However,...', 'rating': 5, 'time_created': '2019-09-12 16:46:10', 'user': {'id': 'so1yeqI9Z2lOs2n_--Vdmw', 'profile_url': 'https://www.yelp.com/user_details?userid=so1yeqI9Z2lOs2n_--Vdmw', 'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/GAcStXOpzSWjJzLf6u8RmQ/o.jpg', 'name': 'Mk K.'}}, {'id': 'bCRuMw4Jpe67m8Hpn21ojQ', 'url': 'https://www.yelp.com/biz/ainslie-brooklyn-2?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=bCRuMw4Jpe67m8Hpn21ojQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'Loved the brick oven pizza in this It

{'reviews': [{'id': 'DBHRxuyFku2jtfYgFBmbLA', 'url': 'https://www.yelp.com/biz/finback-brewery-ridgewood?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=DBHRxuyFku2jtfYgFBmbLA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Food: Unlimited fresh popcorn. I would advise you to grab a bite before coming here because you won't want to leave once you step in. Great delicious beer,...", 'rating': 5, 'time_created': '2019-09-09 16:39:13', 'user': {'id': 'BqXn-1nxi0zqEnPyuCAsXQ', 'profile_url': 'https://www.yelp.com/user_details?userid=BqXn-1nxi0zqEnPyuCAsXQ', 'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/rJmKzmkt7PqOZE1xk6VWFw/o.jpg', 'name': 'Gerry S.'}}, {'id': 'vYdKGJeoEZIa53QHQv3USQ', 'url': 'https://www.yelp.com/biz/finback-brewery-ridgewood?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=vYdKGJeoEZIa53QHQv3USQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Just came back fr

{'reviews': [{'id': 'wnW5b1pKa2I6Bz5I7CsL6Q', 'url': 'https://www.yelp.com/biz/coney-island-brewery-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=wnW5b1pKa2I6Bz5I7CsL6Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'The Merman IPA is 5 out of 5 beers!\nSee Review Below.\n\nLove the citra smell and aroma!\n\nhttps://www.youtube.com/watch?v=Zd2ULCITR_w', 'rating': 5, 'time_created': '2019-07-08 18:23:38', 'user': {'id': 'diJHEmj94Jq9Rnw_09Rtcg', 'profile_url': 'https://www.yelp.com/user_details?userid=diJHEmj94Jq9Rnw_09Rtcg', 'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/_kDEKqn-_ayt-AtrfVZObw/o.jpg', 'name': 'Jeanice C.'}}, {'id': 'IoadAIO65lt9cQZaPm6_Gw', 'url': 'https://www.yelp.com/biz/coney-island-brewery-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=IoadAIO65lt9cQZaPm6_Gw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "The beer here is okay but thi

...

{'reviews': [{'id': 'tETfrA6D7SwybzjmnVZ-gQ', 'url': 'https://www.yelp.com/biz/porto-salvo-bronx?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=tETfrA6D7SwybzjmnVZ-gQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Surprised I haven't left a review. This is like my 4th time here. Seems to get better every time. I can only speak for the brunch menu which is amazing. The...", 'rating': 5, 'time_created': '2019-06-27 15:07:18', 'user': {'id': 'cJclj7pGyIysK-ZYDRzG2w', 'profile_url': 'https://www.yelp.com/user_details?userid=cJclj7pGyIysK-ZYDRzG2w', 'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/MOkpLFArjz8aUKi_Cxf6Sw/o.jpg', 'name': 'Manny C.'}}, {'id': 'lKQSvx_MfOXU5ppA2KdSdg', 'url': 'https://www.yelp.com/biz/porto-salvo-bronx?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=lKQSvx_MfOXU5ppA2KdSdg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Everything I've had there is alw

{'reviews': [{'id': 'P5XAP3KLjZAp0psadywINg', 'url': 'https://www.yelp.com/biz/donovans-reef-sea-bright?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=P5XAP3KLjZAp0psadywINg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "Parking on premises makes it easier for people to enjoy as it's located steps away from their entrance. Lot parking is $10. Arrive early and find street...", 'rating': 5, 'time_created': '2019-08-08 13:09:35', 'user': {'id': 'XM_0xKXd3gdKNaBhXHi4AA', 'profile_url': 'https://www.yelp.com/user_details?userid=XM_0xKXd3gdKNaBhXHi4AA', 'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/i9R-hSTm4b1Bo-2KCpAxsw/o.jpg', 'name': 'Vanesa F.'}}, {'id': 'ld0bAK6vLDOv4FaafXvF_Q', 'url': 'https://www.yelp.com/biz/donovans-reef-sea-bright?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=ld0bAK6vLDOv4FaafXvF_Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'I absolutely love the

{'reviews': [{'id': 'wMYTUmC3L_76YhOY5Y8UAQ', 'url': 'https://www.yelp.com/biz/the-shanty-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=wMYTUmC3L_76YhOY5Y8UAQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'This place is GREAT! The tour and especially the tour guide was extremely informative and she was very nice in dealing with my total ignorance of gin. I...', 'rating': 5, 'time_created': '2019-08-19 08:44:05', 'user': {'id': 'wx4hDn7kl-iePt7dsFTOvg', 'profile_url': 'https://www.yelp.com/user_details?userid=wx4hDn7kl-iePt7dsFTOvg', 'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/3PDIipN7BlKG5DpR_BrhTQ/o.jpg', 'name': 'Raoul T.'}}, {'id': 'aar0jcr4mMjXQBCs_dU6Pg', 'url': 'https://www.yelp.com/biz/the-shanty-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=aar0jcr4mMjXQBCs_dU6Pg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'Attended a fundraising event her

{'reviews': [{'id': '-X1-G0W3btbz2rNU3HWbMQ', 'url': 'https://www.yelp.com/biz/twin-door-tavern-maywood?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=-X1-G0W3btbz2rNU3HWbMQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': "local NJ spot my family's been coming to for years, if we're being honest the food is okay (huuuuge variety i will say, but no one's really going to a...", 'rating': 4, 'time_created': '2019-05-05 10:21:08', 'user': {'id': '2_LjkjNVFhMgBbhP_mBzYQ', 'profile_url': 'https://www.yelp.com/user_details?userid=2_LjkjNVFhMgBbhP_mBzYQ', 'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/5Zr_nTvo5D43sV-VKNscjw/o.jpg', 'name': 'Sara B.'}}, {'id': '3ZEv5TISEvK5uYmA3310-g', 'url': 'https://www.yelp.com/biz/twin-door-tavern-maywood?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=3ZEv5TISEvK5uYmA3310-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw', 'text': 'We used to come here when

KeyboardInterrupt: 

In [19]:
yelp_reviews[0]

{'rest_id': 'gUu-0hyujy5xRM-JAty9JQ',
 'review': {'reviews': [{'id': '8g0IHHggKE2Bq-RNSS283Q',
    'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=8g0IHHggKE2Bq-RNSS283Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=inkVgew63NKO9Oi_a7ADyw',
    'text': 'Love this hidden, off the beaten path gem of a brewery. It gets packed on the weekends and is super lively, but in the best Brooklyn-vibes kind of way....',
    'rating': 5,
    'time_created': '2019-09-07 09:32:40',
    'user': {'id': 'PrioZ6UL7liEWQRmNtgq6w',
     'profile_url': 'https://www.yelp.com/user_details?userid=PrioZ6UL7liEWQRmNtgq6w',
     'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/d96R823jwSwBCBLILpA1mA/o.jpg',
     'name': 'Kaley R.'}},
   {'id': '2MISSNscaoUdpw0nAgMPZg',
    'url': 'https://www.yelp.com/biz/other-half-brewing-brooklyn?adjust_creative=inkVgew63NKO9Oi_a7ADyw&hrid=2MISSNscaoUdpw0nAgMPZg&utm_campaign=yelp_api_v3&utm_me

In [20]:
#This code works
review_list = []
for restaurant in yelp_reviews:
    review_info = {}
    review_info['rest_id'] = restaurant['rest_id']
    for review in restaurant['review']['reviews']:
        review_info['review_id'] = review['id']
        review_info['text'] = review['text']
        review_info['rating'] = review['rating']
        review_info['date'] = review['time_created']
        review_list.append(dict(review_info))

In [ ]:
review_list[0]

In [21]:
review_tuples = []
for review in review_list:
    review_tuple = (review['review_id'], review['text'], review['rating'], review['date'], review['rest_id'])
    review_tuples.append(review_tuple)

In [22]:
stmt = """INSERT INTO reviews (review_id, text, review_rating, date, bus_id) 
            VALUES (%s, %s, %s, %s, %s)"""

cursor.executemany(stmt, review_tuples)
cnx.commit()
cursor.close()
cnx.close()

In [38]:
# yelp_data = all_results(url_params, api_key)

In [40]:
# print(yelp_data[0])

{'error': {'code': 'VALIDATION_ERROR', 'description': '1050 is greater than the maximum of 1000', 'field': 'offset', 'instance': 1050}}


### SQL Questions:
What are the 5 businesses with the highest average ratings?

What are the 5 businesses with the lowest average ratings?

What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?
How many businesses have a rating greater than or 4.5?

How many businesses have a rating less than 3?

Return the text of the reviews for the most reviewed restaurant.

Return the name of the business with the most recent review.

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews.

Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the least reviews.

In [59]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [29]:
#What are the 5 businesses with the highest average ratings?
cursor.execute("""SELECT name, rating
                    FROM yelp_info.businesses
                    WHERE rating = (SELECT MAX(rating) FROM yelp_info.businesses)
                    LIMIT 5""")

for (name, rating) in cursor:
    print(f'{name} has a rating of {rating},')

Ainslie has a rating of 5.0,
Torch & Crown Brewing Company has a rating of 5.0,
Bourbon & Brews has a rating of 5.0,
Davidoff of Geneva Brooklyn has a rating of 5.0,
The Great Northeast Cheese Fest has a rating of 5.0,


In [32]:
#What are the 5 businesses with the lowest average ratings?
cursor.execute("""SELECT name, rating
                    FROM yelp_info.businesses
                    ORDER BY rating ASC
                    LIMIT 5""")

for (name, rating) in cursor:
    print(f'{name} has a rating of {rating},')


El-zahraa Deli, Beer, And Halal Food has a rating of 1.0,
Brownstone Ale has a rating of 1.0,
Long Island Brew House has a rating of 1.5,
Guinness Pub has a rating of 1.5,
BKLYN Beer Garden has a rating of 1.5,


In [38]:
#What is the average rating of restaurants that have a price label of one dollar sign?
cursor.execute("""SELECT ROUND(AVG(rating), 2)
                    FROM yelp_info.businesses
                    WHERE price = '$'""")

for (rating) in cursor:
    print(f'{rating} is the average rating of businesses with one $ sign')

(4.05,) is the average rating of businesses with one $ sign


In [39]:
#What is the average rating of restaurants that have a price label of two dollar signs?
cursor.execute("""SELECT ROUND(AVG(rating), 2)
                    FROM yelp_info.businesses
                    WHERE price = '$$'""")

for (rating) in cursor:
    print(f'{rating} is the average rating of businesses with $$ signs')

(3.87,) is the average rating of businesses with two $ signs


In [40]:
#What is the average rating of restaurants that have a price label of three dollar signs?
cursor.execute("""SELECT ROUND(AVG(rating), 2)
                    FROM yelp_info.businesses
                    WHERE price = '$$$'""")

for (rating) in cursor:
    print(f'{rating} is the average rating of businesses with $$$ signs')

(3.88,) is the average rating of businesses with $$$ signs


In [42]:
#How many businesses have a rating greater than or 4.5?
cursor.execute("""SELECT COUNT(name)
                    FROM yelp_info.businesses
                    WHERE rating >= 4.5""")
for (count) in cursor:
    print(f'There are {count} businesses with a rating greater than 4.5')

There are (258,) businesses with a rating greater than 4.5


In [43]:
#How many businesses have a rating less than 3?
cursor.execute("""SELECT COUNT(name)
                    FROM yelp_info.businesses
                    WHERE rating < 3""")
for (count) in cursor:
    print(f'There are {count} businesses with a rating less than 3')

There are (23,) businesses with a rating less than 3


In [91]:
#Return the text of the reviews for the most reviewed restaurant.

cursor.execute("""SELECT text
                    FROM yelp_info.reviews r
                    JOIN yelp_info.businesses b
                    ON r.bus_id = b.id
                    WHERE b.review_count = (SELECT review_count
                                            FROM yelp_info.businesses
                                            ORDER BY review_count DESC
                                            LIMIT 1)""")
for (text) in cursor:
    print(text)

('Stopped in for lunch today, a three star place with a plus plus for the fourth. Here are the plus plus points:\n- professional staff\n- modern interior\n-...',)
("Cafeteria is my all time favorite restaurant.  I've been going there for almost 20 years.  The food, the decor, the staff, the food...did I mention the...",)
('For starters the atmosphere is very relaxed with a wide variety of music ( went around 3:00am & yea this place is 24 hours! ). Nice restaurant with good...',)


In [92]:
#confirming we're right
cursor.execute("""SELECT text
                    FROM yelp_info.reviews
                    WHERE bus_id LIKE '%HDXLg%'""")

for (text) in cursor:
    print(text)

('Stopped in for lunch today, a three star place with a plus plus for the fourth. Here are the plus plus points:\n- professional staff\n- modern interior\n-...',)
("Cafeteria is my all time favorite restaurant.  I've been going there for almost 20 years.  The food, the decor, the staff, the food...did I mention the...",)
('For starters the atmosphere is very relaxed with a wide variety of music ( went around 3:00am & yea this place is 24 hours! ). Nice restaurant with good...',)


In [55]:
#Return the name of the business with the most recent review.
cursor.execute("""SELECT name
                    FROM yelp_info.businesses bus
                    JOIN yelp_info.reviews rev
                    ON bus.id = rev.bus_id
                    ORDER BY rev.date DESC
                    LIMIT 1""")

for name in cursor:
    print(f'{name} has the most recent review')

("Chumley's",) has the most recent review


In [57]:
#verifying that we got it right
cursor.execute("""SELECT name
                    FROM yelp_info.businesses
                    WHERE id LIKE '%NABjQ'""")

for name in cursor:
    print(name)

("Chumley's",)


In [87]:
#Find the highest rated business and return text of the most recent review. 
# If multiple business have the same rating, select the restaurant with the most reviews.

cursor.execute("""SELECT text
                    FROM yelp_info.reviews rev
                    JOIN yelp_info.businesses bus
                    ON rev.bus_id = bus.id
                    WHERE rev.bus_id = (SELECT id 
                                        FROM yelp_info.businesses
                                        WHERE rating = 5 
                                        ORDER BY review_count DESC
                                        LIMIT 1
                                        )
                    ORDER BY rev.date DESC
                    LIMIT 1;""")
for text in cursor:
    print(text)

("Came here for my birthday. It was a memorable birthday dinner. It was 3 people. It's nestled in a neighborhood. They usually only seat you if everyone is...",)


In [88]:
#Find the lowest rated business and return text of the most recent review. 
#If multiple business have the same rating, select the restaurant with the least reviews.
cursor.execute("""SELECT text
                    FROM yelp_info.reviews rev
                    JOIN yelp_info.businesses bus
                    ON rev.bus_id = bus.id
                    WHERE rev.bus_id = (SELECT id 
                                        FROM yelp_info.businesses
                                        WHERE rating = 1 
                                        ORDER BY review_count ASC
                                        LIMIT 1
                                        )
                    ORDER BY rev.date DESC
                    LIMIT 1;""")
for text in cursor:
    print(text)

("I was hoping it'd be a lot better than it was, but they charge $4 delivery fees and the food tastes as cheap as the prices. There are many other better food...",)
